In [1]:
##### LLAMAPARSE #####
from llama_parse import LlamaParse
from dotenv import load_dotenv


from groq import Groq
from langchain_groq import ChatGroq

#
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()

load_dotenv()

True

In [2]:
parsingInstructionsInterviewer = """
O documento a seguir é um currículode um possível candidato em busca de vagas.
No documento, haverá uma seção contendo as Experiências do candidato e outra contendo as Atividades Extracurriculares dele.
Extraia o que conseguir dessas partes e também as informações pessoais do candidato.
"""

In [3]:
llamaparse = LlamaParse(parsing_instruction=parsingInstructionsInterviewer, result_type="markdown")
parsed_result = llamaparse.get_json_result("CVs\CV Tales Ivalque.pdf")

Started parsing the file under job_id 6e5b9725-2e04-4c4d-bd04-fc567e0d4728


In [4]:
parsed_result

[{'pages': [{'page': 1,
    'text': 'TALES IVALQUE TAVEIRA DE FREITAS\nBrasileiro, 22 anos\n+55 11 98584 0164\ntalesitf@al.insper.edu.br\n\nOBJETIVO\n                           Experiências na área de Dados, Pesquisa e LLMs\n\nFORMAÇÃO\nAgo/21 a Set/26            Insper – Triple Accredited Business School (AACSB, AMBA, EQUIS)\n                           Bacharelado em Engenharia de Computação\n                           Desempenho acima de 80% nos trabalhos e provas\nFev/18 a Jul/21            Colégio Farias Brito – Fortaleza/CE\n                            3º ano Ensino Médio +Curso preparatório ITA/IME;\n                            Competência em cálculo diferencial, matemática e física avançada\n\nJan/16 a Dez/17            Colégio Módulo – Juazeiro do Norte/CE\n                           Ensino Médio\n\nEXPERIENCIA\nOut/21 a Dez/21            Insper --Triple Accredited Business School (AACSB, AMBA, EQUIS)\n                           Desenvolvedor estagiário\n                       

In [5]:
from IPython.display import display, Markdown


display(Markdown(parsed_result[0]['pages'][0]['md']))

**Informações Pessoais:**
- Nome: Tales Ivalque Taveira de Freitas
- Nacionalidade: Brasileiro
- Idade: 22 anos
- Telefone: +55 11 98584 0164
- E-mail: talesitf@al.insper.edu.br

**Objetivo:**
- Experiências na área de Dados, Pesquisa e LLMs

**Formação:**
- **Insper – Triple Accredited Business School (AACSB, AMBA, EQUIS)**
- Bacharelado em Engenharia de Computação
- Período: Ago/21 a Set/26
- Desempenho: Acima de 80% nos trabalhos e provas

- **Colégio Farias Brito – Fortaleza/CE**
- 3º ano Ensino Médio + Curso preparatório ITA/IME
- Período: Fev/18 a Jul/21
- Competência em cálculo diferencial, matemática e física avançada

- **Colégio Módulo – Juazeiro do Norte/CE**
- Ensino Médio
- Período: Jan/16 a Dez/17

**Experiências:**
- **Insper – Triple Accredited Business School (AACSB, AMBA, EQUIS)**
- Cargo: Desenvolvedor estagiário
- Período: Out/21 a Dez/21
- Atividade: Transferência de formato de programação de um dos servidores de ensino da Instituição.

- **Hyperlocal**
- Cargo: Estagiário de Produtos
- Período: Jul/22 a Ago/22
- Atividade: Atuação no planejamento e documentação da API dos principais produtos da empresa.

- **Insper**
- Cargo: NINJA de Ciências dos Dados
- Período: Ago/22 - Presente
- Atividade: Auxílio aos professores das disciplinas na correção de trabalhos e auxílio geral aos alunos da matéria.

**Idiomas:**
- Inglês: Avançado
- Espanhol: Básico

**Informática:**
- Domínio do Pacote Office (Excel intermediário)
- Programação: Python, Arduino, C/C++/C#, Java, Dart

**Atividades Extracurriculares:**
- **Nov/15**: Porta-voz das equipes brasileiras em competição olímpica internacional, realizando tradução em tempo real dos discursos dos representantes das várias equipes participantes.
- **Out/22 - Presente**: Aulas GAS, atuando como professor de revisão de disciplinas pré Pis e PFs.
- **Fev/14 a Dez/18**: Acervo de conquistas em competições acadêmica-científicas, com medalhas de ouro e prata consecutivas em Olimpíadas de Programação, Robótica, Física, Matemática e Química.

In [6]:
chat_model = ChatGroq(temperature=0, model_name="llama-3.1-70b-versatile")

In [7]:
from pydantic import BaseModel, Field
from typing import List

class Activity(BaseModel):
    """
    A detailed description of an activity and the role of a candidate in them.
    """
    description: str = Field(description = "What is the activity")
    role: str = Field(description="Role the candidate served on the acitivity")

class Topics(BaseModel):
    """
    Lists of maybe important topics for a candidate
    """
    expiriences: List[Activity]
    activities: List[Activity]
    projects: List[Activity]


In [8]:
from langchain_core.prompts import ChatPromptTemplate
messages = [
    (
        "system",
        "You are a helpful assistant that hepls identify a candidate's experience, activities and projects based on the following markdown text: {curriculum}. ",
    ),
    (
        "system",
        "Here are the topics:"
    )
]
prompt = ChatPromptTemplate.from_messages(messages)

chat_model = chat_model.with_structured_output(Topics)

chain_structured = prompt | chat_model
response = chain_structured.invoke({"curriculum":parsed_result[0]['pages'][0]['md']})
response

Topics(expiriences=[Activity(description='Desenvolvedor estagiário na Insper – Triple Accredited Business School (AACSB, AMBA, EQUIS)', role='Desenvolvedor estagiário'), Activity(description='Estagiário de Produtos na Hyperlocal', role='Estagiário de Produtos'), Activity(description='NINJA de Ciências dos Dados na Insper', role='NINJA de Ciências dos Dados')], activities=[Activity(description='Porta-voz das equipes brasileiras em competição olímpica internacional, realizando tradução em tempo real dos discursos dos representantes das várias equipes participantes.', role='Porta-voz'), Activity(description='Aulas GAS, atuando como professor de revisão de disciplinas pré Pis e PFs.', role='Professor'), Activity(description='Acervo de conquistas em competições acadêmica-científicas, com medalhas de ouro e prata consecutivas em Olimpíadas de Programação, Robótica, Física, Matemática e Química.', role='Competidor')], projects=[Activity(description='Transferência de formato de programação de 

In [9]:
response.expiriences

[Activity(description='Desenvolvedor estagiário na Insper – Triple Accredited Business School (AACSB, AMBA, EQUIS)', role='Desenvolvedor estagiário'),
 Activity(description='Estagiário de Produtos na Hyperlocal', role='Estagiário de Produtos'),
 Activity(description='NINJA de Ciências dos Dados na Insper', role='NINJA de Ciências dos Dados')]

In [10]:
class Question(BaseModel):
    question1: str = Field(description="First question for the candidate")
    question2: str = Field(description="Second question for the candidate")
    question3: str = Field(description="Third question for the candidate")

In [11]:
llm_questions = ChatGroq(temperature=0.3, model_name="llama-3.1-70b-versatile")

messages = [
    (
        "system",
        """You are a helpful assistant that works in the People Departament and must devise meaninful questions based on a candidate's experience, activities and projects.
           Your main role here is to identify how the candidate responds to different forms of interations, the commitement of the candidate to the activity, and what motivated him for the activities.
           You are to create 3 questions based on the following:
           {activities}
           {experiences}
           {projects}

           Remenber, choose questions for topics with the most importance based on leadership, independence, proactivity, personal relations and challenge overcoming.
        """,
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)

chain_structured = prompt | llm_questions.with_structured_output(Question)

activities = response.activities
experiences = response.expiriences
projects = response.projects


ai_msg = chain_structured.invoke({
    "activities": activities,
    "experiences": experiences,
    "projects": projects
})
ai_msg

Question(question1='Can you describe a situation where you had to translate speeches in real-time as a spokesperson for Brazilian teams in an international Olympic competition? How did you handle the pressure and what skills did you use to ensure accurate communication?', question2='As a competitor in academic-scientific competitions, what motivated you to participate and achieve consecutive gold and silver medals in various fields? How did you balance your time and resources to prepare for these competitions?', question3="In your role as a developer intern at Insper, can you walk us through a challenging project you worked on, such as the transfer of a programming format from one of the institution's servers? How did you approach the problem, and what skills did you use to overcome any obstacles?")

In [12]:
ai_msg.question1

'Can you describe a situation where you had to translate speeches in real-time as a spokesperson for Brazilian teams in an international Olympic competition? How did you handle the pressure and what skills did you use to ensure accurate communication?'

In [13]:
ai_msg.question2

'As a competitor in academic-scientific competitions, what motivated you to participate and achieve consecutive gold and silver medals in various fields? How did you balance your time and resources to prepare for these competitions?'

In [14]:
ai_msg.question3

"In your role as a developer intern at Insper, can you walk us through a challenging project you worked on, such as the transfer of a programming format from one of the institution's servers? How did you approach the problem, and what skills did you use to overcome any obstacles?"